In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

In [2]:
from langchain_groq import ChatGroq

llm = ChatGroq(model="gemma2-9b-it")

In [3]:
import nest_asyncio
nest_asyncio.apply()

In [4]:
connections = {
    "mcpServers": {
        "crypto-price-tracker": {
            "transport": "stdio", 
            "command": "uv",
            "args": [
                "run",
                "D:/My Files/GitHub_Materials/GenAI_Agents/MCP_agents/mcp_server.py"
            ]
        }
    }
}


In [18]:
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.prebuilt import create_react_agent

async def get_crypto_market_agent(query: str):
    client = MultiServerMCPClient(
        connections = {
            "mcpServers": {
                "crypto-price-tracker": {
                    "command": "python",
                      "transport": "stdio",  # or full path to python.exe
                    "args": [
                        "D:/My Files/GitHub_Materials/GenAI_Agents/MCP_agents/mcp_server.py"
                    ]
                }
            }
        }
    )
    tools = await client.get_tools()  # ✅ CORRECT: Await the coroutine

    agent = create_react_agent(
        model=llm,
        tools=tools
    )
    response = await agent.ainvoke(query)
    return response


In [19]:
async def main(crypto_ids,currency: str = "usd"):
    agent = get_crypto_market_agent()
    response = await agent.ainvoke({"crypto_ids":crypto_ids, "currency":currency})
    return response

In [20]:
if __name__ == "__main__":
    import asyncio
    query = "What is the current price of Bitcoin?"
    result = asyncio.run(get_crypto_market_agent(query))
    print(result)

KeyError: 'transport'